### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "German"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = German
Lang Id = 8
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/3-Word Group In Youtube Sentence Firebase"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
#parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Result/2-Word Group Prepare For Youtube Tool"

In [9]:
#df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28 = df_28.head(200)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,ist sie aber so aber was ist mit mir,644,648,und dann ist sie aber so aber was ist mit mir,E5wrYo4PLM4,https://www.youtube.com/watch?v=E5wrYo4PLM4&t=...
1,und nicht auf das was du nicht,391,393,und nicht auf das was du nicht hast,D2EWOivrzz8,https://www.youtube.com/watch?v=D2EWOivrzz8&t=...
2,nicht dass das so was ist das,687,690,oh nee sag nicht dass das so was ist das,m9hZzWA0R3k,https://www.youtube.com/watch?v=m9hZzWA0R3k&t=...
3,das ist ein dass das nicht so,578,580,das ist ein dass das nicht so,XGhAlBQ3gOY,https://www.youtube.com/watch?v=XGhAlBQ3gOY&t=...
4,das ist ja nicht das was ich,548,551,das ist ja nicht das was ich,X0g4THI7vEs,https://www.youtube.com/watch?v=X0g4THI7vEs&t=...
...,...,...,...,...,...,...
195,sie zu mir mit,242,244,nehm sie zu mir mit,TLvLzNIZHU0,https://www.youtube.com/watch?v=TLvLzNIZHU0&t=...
196,dass es so was,712,713,dass es so was gibt,RE5pM9dn5_U,https://www.youtube.com/watch?v=RE5pM9dn5_U&t=...
197,so und das mit,1707,1714,so und das mit,_KwE_S8wdBY,https://www.youtube.com/watch?v=_KwE_S8wdBY&t=...
198,so aber sie ja,191,194,so aber sie ja,EACCbhxJmvs,https://www.youtube.com/watch?v=EACCbhxJmvs&t=...


In [10]:
#df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200 = df_200.head(200)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,oder da würde er sagen da ich glaube es geht l...,1499,1505,oder da würde er sagen da ich glaube es geht l...,L22SNtHYLCw,https://www.youtube.com/watch?v=L22SNtHYLCw&t=...
1,sehen kann dann machen wir alles zu genau also...,2117,2126,sehen kann dann machen wir alles zu genau also...,8BgkmfGO8SI,https://www.youtube.com/watch?v=8BgkmfGO8SI&t=...
2,wie es ihnen gehen muss und was jetzt los ist ...,408,414,wie es ihnen gehen muss und was jetzt los ist ...,3sTLcfBgJh4,https://www.youtube.com/watch?v=3sTLcfBgJh4&t=...
3,ist etwas nicht so wirklich ist und dann mache...,2745,2754,ist etwas nicht so wirklich ist und dann mache...,HWzDSgiLVkI,https://www.youtube.com/watch?v=HWzDSgiLVkI&t=...
4,du denn jetzt machen oder sagen ja was können ...,2046,2051,du denn jetzt machen oder sagen ja was können ...,L22SNtHYLCw,https://www.youtube.com/watch?v=L22SNtHYLCw&t=...
...,...,...,...,...,...,...
195,es muss kein geld sein es kann auch ein danke ...,1095,1099,es muss kein geld sein es kann auch ein danke ...,UoaTpQ3WZIY,https://www.youtube.com/watch?v=UoaTpQ3WZIY&t=...
196,macht aber nichts also wir sehen uns in etwas ...,548,551,macht aber nichts also wir sehen uns in etwas ...,beeJqe8tSV8,https://www.youtube.com/watch?v=beeJqe8tSV8&t=...
197,da hat er gesagt als er dann den hier gesehen ...,776,780,haltenden da hat er gesagt als er dann den hie...,1MND5RM6dOE,https://www.youtube.com/watch?v=1MND5RM6dOE&t=...
198,da machen wir mal wieder auf und wir sehen wie...,982,986,da machen wir mal wieder auf und wir sehen wie...,OLLPmgdUqPc,https://www.youtube.com/watch?v=OLLPmgdUqPc&t=...


In [11]:
#df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000 = df_1000.head(200)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,der noch geld haben will wahrscheinlich eben d...,878,884,der noch geld haben will wahrscheinlich eben d...,e0PyOh2KMQw,https://www.youtube.com/watch?v=e0PyOh2KMQw&t=...
1,aber vielleicht leute zu haben die da sind wo ...,815,824,aber vielleicht leute zu haben die da sind wo ...,e0PyOh2KMQw,https://www.youtube.com/watch?v=e0PyOh2KMQw&t=...
2,ich glaube sie wollte sagen hey du bist nur ei...,3278,3285,ich glaube sie wollte sagen hey du bist nur ei...,twfbyHqyITc,https://www.youtube.com/watch?v=twfbyHqyITc&t=...
3,so stelle ich mir das jetzt vor weil wenn man ...,1803,1810,so stelle ich mir das jetzt vor weil wenn man ...,URWnSf1sEDI,https://www.youtube.com/watch?v=URWnSf1sEDI&t=...
4,das hat damit zu tun dass wir einfach wissen d...,2804,2809,das hat damit zu tun dass wir einfach wissen d...,V1rKsO0okzY,https://www.youtube.com/watch?v=V1rKsO0okzY&t=...
...,...,...,...,...,...,...
195,minuten zeit wir haben eine idee wollen die ih...,231,236,minuten zeit wir haben eine idee wollen die ih...,T89BFtkFLQo,https://www.youtube.com/watch?v=T89BFtkFLQo&t=...
196,machen wir die klappe zu schauen eben nicht me...,763,769,machen wir die klappe zu schauen eben nicht me...,p6Ygs_rqzbc,https://www.youtube.com/watch?v=p6Ygs_rqzbc&t=...
197,bist gerade vater geworden jetzt im glück brin...,589,595,bist gerade vater geworden jetzt im glück brin...,G3Gpb7TiqyE,https://www.youtube.com/watch?v=G3Gpb7TiqyE&t=...
198,wir auch sagen müssen wie wir aber auch viel z...,17,23,wir auch sagen müssen wie wir aber auch viel z...,xXe4nYCSXLY,https://www.youtube.com/watch?v=xXe4nYCSXLY&t=17s


In [12]:
#df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000 = df_5000.head(200)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,lassen wir haben nachher sehr oft da drüber ge...,608,618,ja meine frau hat sich das nie so anmerken las...,rHKdeRoDJDs,https://www.youtube.com/watch?v=rHKdeRoDJDs&t=...
1,einige haben ja ganz schön viel erlebt ich den...,366,381,einige haben ja ganz schön viel erlebt ich den...,_W671aWBZL4,https://www.youtube.com/watch?v=_W671aWBZL4&t=...
2,davon glaube ich geht man so stück für stück w...,1727,1736,davon glaube ich geht man so stück für stück w...,kg23NfjKeTc,https://www.youtube.com/watch?v=kg23NfjKeTc&t=...
3,eine gewisse sicherheit bekommen dass das was ...,135,147,eine gewisse sicherheit bekommen dass das was ...,gweZAS7SiOU,https://www.youtube.com/watch?v=gweZAS7SiOU&t=...
4,und normalerweise bricht meine haut ein paar t...,447,458,ich habe gerade meine periode und normalerweis...,niPfa1UCmaE,https://www.youtube.com/watch?v=niPfa1UCmaE&t=...
...,...,...,...,...,...,...
195,als um rein gekommen bist dass du hast mich so...,3086,3092,als um rein gekommen bist dass du hast mich so...,twfbyHqyITc,https://www.youtube.com/watch?v=twfbyHqyITc&t=...
196,also man kann hier erst mal einen sprung sozus...,149,155,also man kann hier erst mal einen sprung sozus...,yyR3Q1G41t0,https://www.youtube.com/watch?v=yyR3Q1G41t0&t=...
197,okay gratuliere du bist der neue julian gratul...,369,376,okay gratuliere du bist der neue julian gratul...,9sghHS7WDO4,https://www.youtube.com/watch?v=9sghHS7WDO4&t=...
198,falls eure butter gerade aus dem kühlschrank k...,38,44,falls eure butter gerade aus dem kühlschrank k...,YDCh11Mbst0,https://www.youtube.com/watch?v=YDCh11Mbst0&t=38s


In [13]:
#df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000 = df_10000.head(200)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,ja schön danke dir ja das kann ich mir auch gu...,4296,4310,ja schön danke dir ja das kann ich mir auch gu...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
1,die schon lange nicht mehr benutzt wurde und i...,55,65,wie so eine alte kartoffel die schon lange nic...,5b46anicJdI,https://www.youtube.com/watch?v=5b46anicJdI&t=55s
2,es ist auch eine große erleichterung dass wir ...,727,737,es ist auch eine große erleichterung dass wir ...,gweZAS7SiOU,https://www.youtube.com/watch?v=gweZAS7SiOU&t=...
3,ja die frage wird oft gestellt man macht sich ...,1272,1283,ja die frage wird oft gestellt man macht sich ...,rHKdeRoDJDs,https://www.youtube.com/watch?v=rHKdeRoDJDs&t=...
4,emotionale momente wenn so ein brief bei einer...,5,18,emotionale momente wenn so ein brief bei einer...,prJrgq_RXKg,https://www.youtube.com/watch?v=prJrgq_RXKg&t=5s
...,...,...,...,...,...,...
195,nach fast einer woche hat man mich dann nochma...,424,433,nach fast einer woche hat man mich dann nochma...,HI7vQ2yNqBc,https://www.youtube.com/watch?v=HI7vQ2yNqBc&t=...
196,bezahlen kannst gerade so neue geschäfte die d...,430,437,bezahlen kannst gerade so neue geschäfte die d...,HsjHoMpBprs,https://www.youtube.com/watch?v=HsjHoMpBprs&t=...
197,ist kleiner alles ist langsamer du hast einfac...,1325,1335,das bike ist kleiner alles ist langsamer du ha...,5eDsohyyU5w,https://www.youtube.com/watch?v=5eDsohyyU5w&t=...
198,ein kumpel von mir bieten also eigentlich in d...,2230,2237,ein kumpel von mir bieten also eigentlich in d...,twfbyHqyITc,https://www.youtube.com/watch?v=twfbyHqyITc&t=...


In [14]:
#df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000 = df_20000.head(200)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,so dieses ich hab das jetzt verstanden das mac...,3446,3461,so dieses ich hab das jetzt verstanden das mac...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
1,wenn sie das auch selber sehen sie müssten wir...,2328,2340,wenn sie das auch selber sehen sie müssten wir...,tU_GyKFEvsA,https://www.youtube.com/watch?v=tU_GyKFEvsA&t=...
2,wenn ich die konstruktion verstehe das ist ebe...,1468,1481,wenn ich die konstruktion verstehe das ist ebe...,WpcpACck71g,https://www.youtube.com/watch?v=WpcpACck71g&t=...
3,er zu dem zeitpunkt noch ein patient ist dass ...,954,966,er zu dem zeitpunkt noch ein patient ist dass ...,prJrgq_RXKg,https://www.youtube.com/watch?v=prJrgq_RXKg&t=...
4,ja wir freuen uns total mit ihnen dass das gek...,1144,1154,ja wir freuen uns total mit ihnen dass das gek...,rHKdeRoDJDs,https://www.youtube.com/watch?v=rHKdeRoDJDs&t=...
...,...,...,...,...,...,...
195,den menschen in den mittelpunkt stellen was an...,78,87,den menschen in den mittelpunkt stellen was an...,8hWALUSrbGU,https://www.youtube.com/watch?v=8hWALUSrbGU&t=78s
196,wie gesagt hier bitte ein gutes vorbild sein d...,414,423,wie gesagt hier bitte ein gutes vorbild sein d...,cQgm1KUh9AA,https://www.youtube.com/watch?v=cQgm1KUh9AA&t=...
197,unerwartet was einen da wahrscheinlich treffen...,3697,3704,unerwartet was einen da wahrscheinlich treffen...,k28VLpf7gjU,https://www.youtube.com/watch?v=k28VLpf7gjU&t=...
198,bis man zu einer gemeinsamen stellungnahme kom...,498,508,bis man zu einer gemeinsamen stellungnahme kom...,6I3nbZXZ4WM,https://www.youtube.com/watch?v=6I3nbZXZ4WM&t=...


In [15]:
#df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000 = df_40000.head(200)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,doch noch bevor ich antworten konnte schrie si...,358,366,doch noch bevor ich antworten konnte schrie si...,9k2_UwkcYWo,https://www.youtube.com/watch?v=9k2_UwkcYWo&t=...
1,so dieses ich hab das jetzt verstanden das mac...,3446,3461,so dieses ich hab das jetzt verstanden das mac...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
2,nun noch einmal ganz kurz ich glaube nicht all...,4052,4065,nun noch einmal ganz kurz ich glaube nicht all...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
3,die fragen sind nicht so willkürlich wie bei e...,2515,2525,die fragen sind nicht so willkürlich wie bei e...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
4,mal zurück zu ihrer tätigkeit als solche mich ...,758,774,mal zurück zu ihrer tätigkeit als solche mich ...,URZkhWJpxs4,https://www.youtube.com/watch?v=URZkhWJpxs4&t=...
...,...,...,...,...,...,...
195,universum diese schulden wir haben jetzt gerad...,690,696,universum diese schulden wir haben jetzt gerad...,hm9J96RFCfM,https://www.youtube.com/watch?v=hm9J96RFCfM&t=...
196,wie schaffen sie denn transparenz für ihre arb...,940,949,wie schaffen sie denn transparenz für ihre arb...,G_NWbNhbeNc,https://www.youtube.com/watch?v=G_NWbNhbeNc&t=...
197,klar beide seiten werden irgendwie zufriedener...,3407,3417,klar beide seiten werden irgendwie zufriedener...,ffP5IB9bPgQ,https://www.youtube.com/watch?v=ffP5IB9bPgQ&t=...
198,er hat jetzt auch einfach hinter also ich hab ...,453,460,er hat jetzt auch einfach hinter also ich hab ...,LmauHg9aoRY,https://www.youtube.com/watch?v=LmauHg9aoRY&t=...


#### Sending Data To Firestore

##### Language Data

In [16]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [17]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [18]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [19]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [20]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [21]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [22]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [23]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [24]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [25]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [26]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [27]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [28]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [29]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [30]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [31]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [32]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [33]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [34]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [35]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [36]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [37]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [38]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [39]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [40]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [41]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [42]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [43]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [44]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [45]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [46]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [47]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [48]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1675249559
  nanos: 326195000
}